In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/chess-pieces-detection-images-dataset/Rook-resize/00000137_resized.jpg
/kaggle/input/chess-pieces-detection-images-dataset/Rook-resize/00000010_resized.jpg
/kaggle/input/chess-pieces-detection-images-dataset/Rook-resize/00000067_resized.jpg
/kaggle/input/chess-pieces-detection-images-dataset/Rook-resize/00000060_resized.jpg
/kaggle/input/chess-pieces-detection-images-dataset/Rook-resize/00000210_resized.jpg
/kaggle/input/chess-pieces-detection-images-dataset/Rook-resize/00000240_resized.jpg
/kaggle/input/chess-pieces-detection-images-dataset/Rook-resize/00000117_resized.jpg
/kaggle/input/chess-pieces-detection-images-dataset/Rook-resize/00000130_resized.jpg
/kaggle/input/chess-pieces-detection-images-dataset/Rook-resize/00000005_resized.jpg
/kaggle/input/chess-pieces-detection-images-dataset/Rook-resize/00000073_resized.jpg
/kaggle/input/chess-pieces-detection-images-dataset/Rook-resize/00000023_resized.jpg
/kaggle/input/chess-pieces-detection-images-dataset/Rook-resize/0

In [2]:
import tensorflow
from tensorflow import keras
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

In [3]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.25,
        zoom_range=0.25,
        horizontal_flip=True,
        height_shift_range=0.5,
        validation_split=0.2
    )



train_generator = train_datagen.flow_from_directory(
        '/kaggle/input/chess-pieces-detection-images-dataset/',
        target_size=(224, 224),
        class_mode='categorical',
        batch_size=32,
        shuffle=True,
        subset='training'
    )
    
validation_generator = train_datagen.flow_from_directory(
        '/kaggle/input/chess-pieces-detection-images-dataset/',
        target_size=(224, 224),
        class_mode='categorical',
        batch_size=32,
        shuffle=True,
        subset='validation'
    )

Found 523 images belonging to 5 classes.
Found 128 images belonging to 5 classes.


In [4]:
from keras import Sequential , layers

In [5]:
from keras.applications.vgg16 import VGG16
vgg = VGG16()

2022-03-07 11:25:02.889194: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-07 11:25:02.996678: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-07 11:25:02.997344: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-07 11:25:02.999197: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

553476096/553467096 [==============================] - 3s 0us/step


In [6]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [7]:
model = Sequential()

for layer in vgg.layers[:-1]:
    model.add(layer)
    
    

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

In [9]:
for i in model.layers:
    i.trainable = False
    

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

In [11]:
from keras.layers import Dense
model.add(Dense(5,activation = 'softmax'))

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

In [13]:
model.compile(optimizer = 'Adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

In [14]:
history=model.fit_generator(train_generator,steps_per_epoch=train_generator.samples//train_generator.batch_size,epochs=20,validation_data=validation_generator,validation_steps=validation_generator.samples//validation_generator.batch_size)

/opt/conda/lib/python3.7/site-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
2022-03-07 11:25:09.653332: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/20


2022-03-07 11:25:12.105068: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


16/16 [==============================] - 21s 726ms/step - loss: 1.8447 - accuracy: 0.2261 - val_loss: 1.5467 - val_accuracy: 0.3281
Epoch 2/20
16/16 [==============================] - 8s 504ms/step - loss: 1.5343 - accuracy: 0.3585 - val_loss: 1.4143 - val_accuracy: 0.5547
Epoch 3/20
16/16 [==============================] - 7s 467ms/step - loss: 1.4429 - accuracy: 0.3951 - val_loss: 1.2819 - val_accuracy: 0.5859
Epoch 4/20
16/16 [==============================] - 7s 448ms/step - loss: 1.3468 - accuracy: 0.4521 - val_loss: 1.4502 - val_accuracy: 0.3984
Epoch 5/20
16/16 [==============================] - 7s 455ms/step - loss: 1.3381 - accuracy: 0.4542 - val_loss: 1.2258 - val_accuracy: 0.5391
Epoch 6/20
16/16 [==============================] - 8s 483ms/step - loss: 1.2722 - accuracy: 0.4725 - val_loss: 1.1790 - val_accuracy: 0.5156
Epoch 7/20
16/16 [==============================] - 7s 470ms/step - loss: 1.2242 - accuracy: 0.5438 - val_loss: 1.1788 - val_accuracy: 0.4844
Epoch 8/20
16/16